In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pickle import dump, load

In [2]:
movie_data = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
movie_data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [3]:
credit_data = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
credit_data

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
movie_data = movie_data.drop('budget', axis=1)
movie_data = movie_data.drop('homepage', axis=1)
movie_data = movie_data.drop('original_language', axis=1)
movie_data = movie_data.drop('original_title', axis=1)
movie_data = movie_data.drop('popularity', axis=1)
movie_data = movie_data.drop('production_companies', axis=1)
movie_data = movie_data.drop('production_countries', axis=1)
movie_data = movie_data.drop('release_date', axis=1)
movie_data = movie_data.drop('revenue', axis=1)
movie_data = movie_data.drop('runtime', axis=1)
movie_data = movie_data.drop('spoken_languages', axis=1)
movie_data = movie_data.drop('status', axis=1)
movie_data = movie_data.drop('tagline', axis=1)
movie_data = movie_data.drop('vote_average', axis=1)
movie_data = movie_data.drop('vote_count', axis=1)
movie_data

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter
...,...,...,...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,El Mariachi
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",72766,[],A newlywed couple's honeymoon is upended by th...,Newlyweds
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered"
4801,[],126186,[],When ambitious New York attorney Sam is sent t...,Shanghai Calling


In [5]:
import sqlite3

os.makedirs('./data', exist_ok=True)
conn = sqlite3.connect("./data/movies_database.db")

movie_data.to_sql("movies_table", conn, if_exists = "replace", index = False)
credit_data.to_sql("credits_table", conn, if_exists = "replace", index = False)


4803

In [6]:
query = """
    SELECT * FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title
"""

In [7]:
total_data = pd.read_sql_query(query, conn)
conn.close()


In [8]:
total_data.head()

,genres,id,keywords,overview,title,movie_id,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
import json

def load_json_safe(json_str, default_value = None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value
    
total_data["genres"] = total_data["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
total_data["keywords"] = total_data["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

total_data["cast"] = total_data["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

total_data["crew"] = total_data["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))

total_data["overview"] = total_data["overview"].apply(lambda x: [x])

total_data.head()

,genres,id,keywords,overview,title,movie_id,title,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","[In the 22nd century, a paraplegic Marine is d...",Avatar,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","[Captain Barbossa, long believed to be dead, h...",Pirates of the Caribbean: At World's End,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",[A cryptic message from Bond’s past sends him ...,Spectre,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",[Following the death of District Attorney Harv...,The Dark Knight Rises,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","[John Carter is a war-weary, former military c...",John Carter,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton


In [10]:
total_data["overview"] = total_data["overview"].apply(lambda x: [str(x)])
total_data["genres"] = total_data["genres"].apply(lambda x: [str(genre) for genre in x])
total_data["keywords"] = total_data["keywords"].apply(lambda x: [str(keyword) for keyword in x])
total_data["cast"] = total_data["cast"].apply(lambda x: [str(actor) for actor in x])
total_data["crew"] = total_data["crew"].apply(lambda x: [str(crew_member) for crew_member in x])

total_data["tags"] = total_data["overview"] + total_data["genres"] + total_data["keywords"] + total_data["cast"] + total_data["crew"]
total_data["tags"] = total_data["tags"].apply(lambda x: ",".join(x).replace(",", " "))

total_data.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

total_data = total_data.T.drop_duplicates().T

total_data

,id,title,movie_id,tags
0,19995,Avatar,19995,['In the 22nd century a paraplegic Marine is ...
1,285,Pirates of the Caribbean: At World's End,285,['Captain Barbossa long believed to be dead ...
2,206647,Spectre,206647,['A cryptic message from Bond’s past sends him...
3,49026,The Dark Knight Rises,49026,"[""Following the death of District Attorney Har..."
4,49529,John Carter,49529,"[""John Carter is a war-weary former military ..."
...,...,...,...,...
4804,9367,El Mariachi,9367,['El Mariachi just wants to play his guitar an...
4805,72766,Newlyweds,72766,"[""A newlywed couple's honeymoon is upended by ..."
4806,231617,"Signed, Sealed, Delivered",231617,"['""Signed Sealed Delivered"" introduces a ded..."
4807,126186,Shanghai Calling,126186,['When ambitious New York attorney Sam is sent...


In [11]:
y = total_data['title']
X = total_data['tags']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train

402     ["When Travis  the mouthy son of a criminal  d...
4543    ['A struggling young musician and devoted fan ...
3541    ["On the hottest day of the year on a street i...
2377    ['A father and son go on the run after the dad...
4584    ["Dogtown and Z-Boys follows the evolution of ...
                              ...                        
4426    ['Three childhood friends set aside their pers...
466     ['Hoping to alter the events of the past  a 19...
3092    ['Krishna (Roshan) is born with magical powers...
3772    ['A young woman  recently released from a ment...
860     ["The Burlesque Lounge has its best days behin...
Name: tags, Length: 3847, dtype: object

In [12]:
vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(total_data["tags"])

# model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
# model.fit(tfidf_matrix)

# def get_movie_recommendations(movie_title):
#     movie_index = total_data[total_data["title"] == movie_title].index[0]
#     distances, indices = model.kneighbors(tfidf_matrix[movie_index])
#     similar_movies = [(total_data["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
#     return similar_movies[1:]

# input_movie = "How to Train Your Dragon"
# recommendations = get_movie_recommendations(input_movie)
# print("Film recommendations '{}'".format(input_movie))
# for movie, distance in recommendations:
#     print("- Film: {}".format(movie))

Film recommendations 'How to Train Your Dragon'
- Film: How to Train Your Dragon 2
- Film: Dragon Nest: Warriors' Dawn
- Film: Pete's Dragon
- Film: George and the Dragon
- Film: Eragon


In [14]:
print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 279165 stored elements and shape (4809, 27262)>
  Coords	Values
  (0, 12180)	0.03151908290978514
  (0, 24406)	0.048379860150793964
  (0, 226)	0.16639344638453424
  (0, 4244)	0.09972758681715516
  (0, 17929)	0.16253818441045395
  (0, 15202)	0.23137799054590646
  (0, 12723)	0.038025277919232214
  (0, 7086)	0.15173657303042123
  (0, 24666)	0.026756153759407666
  (0, 16253)	0.12850597838633304
  (0, 17872)	0.1564550323463162
  (0, 17415)	0.041292996146924
  (0, 25519)	0.1315319027512186
  (0, 16055)	0.08914397116687293
  (0, 3643)	0.05348845148703126
  (0, 2392)	0.08333651268869367
  (0, 24764)	0.12714128563809018
  (0, 2646)	0.08705899307416562
  (0, 9424)	0.1168750293971056
  (0, 17519)	0.13816431682666042
  (0, 1146)	0.05439267362118873
  (0, 19261)	0.1446217335018981
  (0, 1114)	0.04560222243213593
  (0, 923)	0.2920129866876464
  (0, 4683)	0.13816431682666042
  :	:
  (4808, 17249)	0.0932401871112217
  (4808, 7460)	0.16569599

In [15]:
# from sklearn.neighbors import KNeighborsClassifier

# model = KNeighborsClassifier(metric='cosine')
# model.fit(X_train, y_train)



KNeighborsClassifier(metric='cosine')

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(tfidf_matrix)

def recommend(movie):
    movie_index = total_data[total_data["title"] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(total_data.iloc[i[0]].title)

recommend("How to Train Your Dragon")



How to Train Your Dragon 2
Dragon Nest: Warriors' Dawn
Pete's Dragon
George and the Dragon
Eragon


array([[1.        , 0.03227651, 0.01652419, ..., 0.02344163, 0.03698118,
        0.00662843],
       [0.03227651, 1.        , 0.02964415, ..., 0.03290194, 0.02108433,
        0.01459805],
       [0.01652419, 0.02964415, 1.        , ..., 0.02761013, 0.0278829 ,
        0.01273203],
       ...,
       [0.02344163, 0.03290194, 0.02761013, ..., 1.        , 0.0404121 ,
        0.0325674 ],
       [0.03698118, 0.02108433, 0.0278829 , ..., 0.0404121 , 1.        ,
        0.03639584],
       [0.00662843, 0.01459805, 0.01273203, ..., 0.0325674 , 0.03639584,
        1.        ]], shape=(4809, 4809))